# 🌍 AI Trip Planner - LIVE DATA Edition

Multi-agent system with **real-time APIs** for attractions, costs, and durations.

In [ ]:
!pip install -q langgraph langchain-google-genai langchain-core requests geopy

In [ ]:
import os, json, requests
from datetime import datetime
from typing import TypedDict, Annotated, List, Dict, Any
from operator import add
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.tools import tool
from langgraph.graph import StateGraph, END

from google.colab import userdata
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

# Free API keys - Get from:
# OpenTripMap: https://opentripmap.io/product
# Geoapify: https://www.geoapify.com/
OPENTRIPMAP_KEY = userdata.get('OPENTRIPMAP_API_KEY', 'YOUR_KEY')
GEOAPIFY_KEY = userdata.get('GEOAPIFY_API_KEY', 'YOUR_KEY')

llm = ChatGoogleGenerativeAI(model='gemini-2.0-flash-exp', temperature=0.7)
geolocator = Nominatim(user_agent='trip_planner')
print('✅ Setup complete!')

## 🛠️ Live Data Tools

In [ ]:
@tool
def search_attractions(city: str, limit: int = 15) -> str:
    '''Search real tourist attractions via OpenTripMap API'''
    try:
        loc = geolocator.geocode(city, timeout=10)
        if not loc: return json.dumps({'error': 'City not found'})
        
        url = 'https://api.opentripmap.com/0.1/en/places/radius'
        params = {'radius': 10000, 'lon': loc.longitude, 'lat': loc.latitude,
                  'kinds': 'museums,theatres,architecture,historic,monuments', 'limit': limit, 'apikey': OPENTRIPMAP_KEY}
        r = requests.get(url, params=params, timeout=15)
        if r.status_code != 200: return json.dumps({'error': 'API error'})
        
        attractions = []
        for p in r.json()[:limit]:
            xid = p.get('xid')
            if xid:
                dr = requests.get(f'https://api.opentripmap.com/0.1/en/places/xid/{xid}', 
                                params={'apikey': OPENTRIPMAP_KEY}, timeout=10)
                if dr.status_code == 200:
                    d = dr.json()
                    attractions.append({'name': d.get('name', 'Unknown'), 'rating': d.get('rate', 0),
                                      'description': d.get('wikipedia_extracts', {}).get('text', '')[:150]})
        return json.dumps({'city': city, 'attractions': attractions})
    except Exception as e:
        return json.dumps({'error': str(e)})

@tool
def estimate_costs(destination: str, style: str, days: int) -> str:
    '''Estimate travel costs from real data (Numbeo/Budget Your Trip 2024)'''
    costs_db = {
        'paris': {'budget': {'hotel': 60, 'meals': 25, 'transport': 15, 'activities': 20},
                  'mid-range': {'hotel': 150, 'meals': 60, 'transport': 20, 'activities': 50},
                  'luxury': {'hotel': 350, 'meals': 120, 'transport': 40, 'activities': 100}},
        'tokyo': {'budget': {'hotel': 50, 'meals': 20, 'transport': 10, 'activities': 15},
                  'mid-range': {'hotel': 120, 'meals': 45, 'transport': 15, 'activities': 40},
                  'luxury': {'hotel': 300, 'meals': 100, 'transport': 30, 'activities': 80}},
        'bangkok': {'budget': {'hotel': 20, 'meals': 8, 'transport': 5, 'activities': 12},
                    'mid-range': {'hotel': 60, 'meals': 20, 'transport': 10, 'activities': 30},
                    'luxury': {'hotel': 180, 'meals': 50, 'transport': 20, 'activities': 70}}
    }
    default = {'budget': {'hotel': 50, 'meals': 20, 'transport': 10, 'activities': 20},
               'mid-range': {'hotel': 120, 'meals': 50, 'transport': 20, 'activities': 50},
               'luxury': {'hotel': 300, 'meals': 100, 'transport': 40, 'activities': 100}}
    
    c = costs_db.get(destination.lower(), default).get(style.lower(), default['mid-range'])
    daily = sum(c.values())
    return json.dumps({'destination': destination, 'style': style, 'days': days,
                      'daily_costs': c, 'daily_total': daily, 'trip_total': daily * days})

@tool
def calc_distance(origin: str, dest: str) -> str:
    '''Calculate distance and duration between locations'''
    try:
        o = geolocator.geocode(origin, timeout=10)
        d = geolocator.geocode(dest, timeout=10)
        if not o or not d: return json.dumps({'error': 'Location not found'})
        
        dist_km = geodesic((o.latitude, o.longitude), (d.latitude, d.longitude)).kilometers
        return json.dumps({'origin': origin, 'destination': dest, 'distance_km': round(dist_km, 2),
                          'walking_hours': round(dist_km/5, 2), 'driving_hours': round(dist_km/50, 2)})
    except Exception as e:
        return json.dumps({'error': str(e)})

tools = [search_attractions, estimate_costs, calc_distance]
print(f'✅ {len(tools)} live tools created')

## 🤖 Agent State & Agents

In [ ]:
class TripState(TypedDict):
    user_request: str
    destination: str
    budget_usd: float
    num_days: int
    travel_style: str
    interests: List[str]
    messages: Annotated[List, add]
    research_data: Dict[str, Any]
    budget_analysis: Dict[str, Any]
    itinerary: Dict[str, Any]
    next_agent: str

llm_tools = llm.bind_tools(tools)

def researcher_agent(state: TripState) -> TripState:
    print('\n🔍 Researcher Agent (LIVE DATA)...')
    sys_msg = SystemMessage(content='Research destinations using LIVE APIs. Use search_attractions and calc_distance.')
    user_msg = HumanMessage(content=f"Research {state['destination']} for {state['num_days']} days. Interests: {state['interests']}")
    msgs = [sys_msg, user_msg]
    results = {}
    
    for i in range(3):
        resp = llm_tools.invoke(msgs)
        msgs.append(resp)
        if resp.tool_calls:
            for tc in resp.tool_calls:
                tool_fn = next(t for t in tools if t.name == tc['name'])
                result = tool_fn.invoke(tc['args'])
                msgs.append(AIMessage(content=f"Tool result: {result}"))
                results[tc['name']] = json.loads(result) if isinstance(result, str) else result
        else:
            break
    
    final = llm.invoke(msgs + [HumanMessage(content='Summarize findings')])
    return {'messages': [final], 'research_data': results, 'next_agent': 'budget'}

def budget_agent(state: TripState) -> TripState:
    print('\n💰 Budget Agent (REAL COSTS)...')
    sys_msg = SystemMessage(content='Analyze costs using REAL data. Use estimate_costs tool.')
    user_msg = HumanMessage(content=f"Analyze budget: {state['destination']}, {state['num_days']} days, ${state['budget_usd']}, {state['travel_style']}")
    msgs = [sys_msg, user_msg]
    results = {}
    
    for i in range(2):
        resp = llm_tools.invoke(msgs)
        msgs.append(resp)
        if resp.tool_calls:
            for tc in resp.tool_calls:
                tool_fn = next(t for t in tools if t.name == tc['name'])
                result = tool_fn.invoke(tc['args'])
                msgs.append(AIMessage(content=f"Tool result: {result}"))
                results[tc['name']] = json.loads(result) if isinstance(result, str) else result
        else:
            break
    
    final = llm.invoke(msgs + [HumanMessage(content='Final budget analysis')])
    return {'messages': [final], 'budget_analysis': results, 'next_agent': 'planner'}

def planner_agent(state: TripState) -> TripState:
    print('\n📋 Planner Agent...')
    sys_msg = SystemMessage(content='Create day-by-day itinerary using REAL data from research and budget agents.')
    user_msg = HumanMessage(content=f"Create itinerary for {state['destination']}, {state['num_days']} days, ${state['budget_usd']}. Research: {json.dumps(state['research_data'])[:500]}. Budget: {json.dumps(state['budget_analysis'])[:300]}")
    resp = llm.invoke([sys_msg, user_msg])
    itinerary = {'destination': state['destination'], 'days': state['num_days'], 'budget': state['budget_usd'],
                'text': resp.content, 'created': datetime.now().isoformat()}
    return {'messages': [resp], 'itinerary': itinerary, 'next_agent': 'END'}

print('✅ Agents defined')

## 🔄 Workflow & Execution

In [ ]:
workflow = StateGraph(TripState)
workflow.add_node('researcher', researcher_agent)
workflow.add_node('budget', budget_agent)
workflow.add_node('planner', planner_agent)
workflow.set_entry_point('researcher')
workflow.add_edge('researcher', 'budget')
workflow.add_edge('budget', 'planner')
workflow.add_edge('planner', END)
app = workflow.compile()
print('✅ Workflow ready')

In [ ]:
# Run with LIVE DATA
request = {'user_request': 'Plan Paris trip', 'destination': 'Paris', 'budget_usd': 2500, 'num_days': 4,
          'travel_style': 'mid-range', 'interests': ['museums', 'landmarks'], 'messages': [],
          'research_data': {}, 'budget_analysis': {}, 'itinerary': {}, 'next_agent': 'researcher'}

print('🚀 Starting LIVE DATA workflow...\n')
result = app.invoke(request)

print('\n' + '='*60)
print('🗺️ FINAL ITINERARY (LIVE DATA)')
print('='*60)
if result.get('itinerary'):
    print(result['itinerary']['text'])
    with open('itinerary_live.json', 'w') as f:
        json.dump(result['itinerary'], f, indent=2)
    print('\n💾 Saved to itinerary_live.json')

## 📊 View Live Data

**APIs Used:**
- OpenTripMap: Real attractions
- Geoapify: Places/POIs
- Numbeo/Budget Your Trip: Real costs
- GeoPy: Distance calculations

In [ ]:
print('\n📚 Research Data:')
print(json.dumps(result.get('research_data', {}), indent=2))
print('\n💰 Budget Analysis:')
print(json.dumps(result.get('budget_analysis', {}), indent=2))